In [306]:
from gc_utils import *

alg = Algebra(4)
ga2 = Algebra(2)
locals().update(alg.blades)

In [307]:
# 2d general linear transform
def random_linear_trans(alg: Algebra):
    matrix = np.random.rand(alg.d, alg.d)
    return matrix2trans(matrix, alg)

def random_ortho_trans(alg: Algebra):
    from scipy.stats import ortho_group
    matrix = ortho_group.rvs(dim=alg.d)
    return matrix2trans(matrix, alg)

f = random_linear_trans(ga2)
x = random_r_blade(1, ga2)
f(x)

0.808 𝐞₁ + 0.399 𝐞₂

In [308]:
# 2d polar decomposition
# f = SUU
df = derivative(f, x, ga2, grade=1)
UU = df / norm(df)
S = lambda x: UU * f(x)
S(x)*UU, f(x)

(0.808 𝐞₁ + 0.399 𝐞₂, 0.808 𝐞₁ + 0.399 𝐞₂)

In [309]:
# UU is rotor, S is symmetric
UU, curl(S, x, ga2, grade=1)

(0.844 + -0.536 𝐞₁₂, 8.33e-11 𝐞₁₂)

In [310]:
#6.3 remove the condition of 2 dimension
a, b, x = random_r_vectors(3, alg)
g = lambda x: x + (x|a)*b
abframe = [a,b]
r_frame = reciprocal(abframe)
dg = 2 + a*b
derivative(g, x, alg, grade=1, frame=abframe, r_frame=r_frame), dg

(2.55 + 0.0263 𝐞₁₂ + -0.0618 𝐞₁₃ + -0.0671 𝐞₁₄ + -0.0808 𝐞₂₃ + -0.111 𝐞₂₄ + 0.0539 𝐞₃₄ + -2.67e-16 𝐞₁₂₃₄,
 2.55 + 0.0263 𝐞₁₂ + -0.0618 𝐞₁₃ + -0.0671 𝐞₁₄ + -0.0808 𝐞₂₃ + -0.111 𝐞₂₄ + 0.0539 𝐞₃₄)

In [311]:
#6.3
dg_norm = norm(dg)
UU = dg / dg_norm
U = simple_rotor_sqrt(UU)
S = lambda x: U.sw(g(x))
curl(S, x, alg, grade=1), U.reverse().sw(S(x)), g(x)

(-1.11e-10 𝐞₁₂ + 9.71e-11 𝐞₁₃ + -4.16e-10 𝐞₁₄ + 1.11e-10 𝐞₂₃ + 8.33e-11 𝐞₂₄ + -4.16e-11 𝐞₃₄,
 1.04 𝐞₁ + 0.74 𝐞₂ + 0.122 𝐞₃ + 0.259 𝐞₄,
 1.04 𝐞₁ + 0.74 𝐞₂ + 0.122 𝐞₃ + 0.259 𝐞₄)

In [312]:
#6.6
frame = alg.frame
i, j = 1, 3
aij = (1/np.sqrt(2))*(frame[i] + frame[j])
[aij.sw(x) for x in frame]

[-1.0 𝐞₁, 1.0 𝐞₄, -1.0 𝐞₃, 1.0 𝐞₂]

In [313]:
#6.7
alpha = 3
Sij = lambda x: x + (alpha * (x|frame[i]) * frame[j])
[Sij(x) for x in frame]

[1 𝐞₁, 1 𝐞₂ + 3 𝐞₄, 1 𝐞₃, 1 𝐞₄]

In [314]:
#7.2
g = random_linear_trans(alg)
g_ = lambda x, y: x | g(y)
derivative(lambda x: g_(x, b), x, alg, grade=1), g(b)

(0.628 𝐞₁ + 0.705 𝐞₂ + 1.14 𝐞₃ + 0.255 𝐞₄,
 0.628 𝐞₁ + 0.705 𝐞₂ + 1.14 𝐞₃ + 0.255 𝐞₄)

In [315]:
# check nonsingular
outermorphism(g, alg.pseudoscalar((1,)), alg)

-0.189 𝐞₁₂₃₄

In [316]:
#7.5 orthogonal transformation is its isometry
f = random_ortho_trans(alg)
x | f(b), f(x) | f(f(b))

(-0.211, -0.211)

In [317]:
# skew bilinear form
gm = skew_part(g, alg)
G = simplicial_derivative(lambda vectors: vectors[0] | gm(vectors[1]), np.zeros(2), alg)
x | gm(b), x | (b | G)

(0.0157, 0.0157)

In [318]:
#7.6b f is not an isometry of gm
adjoint_outermorphism(f, G, alg), G

(0.26 𝐞₁₂ + -0.364 𝐞₁₃ + -0.135 𝐞₁₄ + -0.328 𝐞₂₃ + 0.0145 𝐞₂₄ + 0.127 𝐞₃₄,
 0.309 𝐞₁₂ + 0.342 𝐞₁₃ + -0.269 𝐞₁₄ + -0.0283 𝐞₂₃ + -0.00432 𝐞₂₄ + 0.238 𝐞₃₄)

In [319]:
#7.8 the inverse of gm
blades = bivector_split(G, alg)
gm(sum(-x | blade/normsq(blade) for blade in blades)), x

(0.852 𝐞₁ + 0.509 𝐞₂ + 0.0745 𝐞₃ + 0.0402 𝐞₄,
 0.852 𝐞₁ + 0.509 𝐞₂ + 0.0745 𝐞₃ + 0.0402 𝐞₄)

In [320]:
#7.12 assume splitted blades are normal
blades_norm = [norm(b) for b in blades]
blades = [b / n for b, n in zip(blades, blades_norm)]
gprod(blades)

3.05e-16 + 5.55e-17 𝐞₁₄ + 4.16e-17 𝐞₂₃ + 2.08e-17 𝐞₂₄ + 1.39e-17 𝐞₃₄ + 1.0 𝐞₁₂₃₄

In [321]:
#7.17
theta = np.random.rand(2)
rotors = [blade_exp(t*blade) for t, blade in zip(theta, blades)]
[(U.sw(G), G) for U in rotors for G in blades]

[(0.299 𝐞₁₂ + -0.158 𝐞₁₃ + 0.0773 𝐞₁₄ + -0.495 𝐞₂₃ + -0.645 𝐞₂₄ + 0.468 𝐞₃₄,
  0.299 𝐞₁₂ + -0.158 𝐞₁₃ + -0.495 𝐞₂₃ + 0.0773 𝐞₁₄ + -0.645 𝐞₂₄ + 0.468 𝐞₃₄),
 (0.468 𝐞₁₂ + 0.645 𝐞₁₃ + -0.495 𝐞₁₄ + 0.0773 𝐞₂₃ + 0.158 𝐞₂₄ + 0.299 𝐞₃₄,
  0.468 𝐞₁₂ + 0.645 𝐞₁₃ + 0.0773 𝐞₂₃ + -0.495 𝐞₁₄ + 0.158 𝐞₂₄ + 0.299 𝐞₃₄),
 (0.299 𝐞₁₂ + -0.158 𝐞₁₃ + 0.0773 𝐞₁₄ + -0.495 𝐞₂₃ + -0.645 𝐞₂₄ + 0.468 𝐞₃₄,
  0.299 𝐞₁₂ + -0.158 𝐞₁₃ + -0.495 𝐞₂₃ + 0.0773 𝐞₁₄ + -0.645 𝐞₂₄ + 0.468 𝐞₃₄),
 (0.468 𝐞₁₂ + 0.645 𝐞₁₃ + -0.495 𝐞₁₄ + 0.0773 𝐞₂₃ + 0.158 𝐞₂₄ + 0.299 𝐞₃₄,
  0.468 𝐞₁₂ + 0.645 𝐞₁₃ + 0.0773 𝐞₂₃ + -0.495 𝐞₁₄ + 0.158 𝐞₂₄ + 0.299 𝐞₃₄)]

In [322]:
#7.18
first_vecs = [blade_split(blade, alg)[0] for blade in blades]
first_vecs = [v/norm(v) for v in first_vecs]
second_vecs = [v | blade for v, blade in zip(first_vecs, blades)]
inter_plane_rotors = [simple_rotor_sqrt(a * b) for a, b in (first_vecs, second_vecs)]
U = gprod(inter_plane_rotors)
terms_ratio(U.sw(blades[0]), blades[1])

array([1., 1., 1., 1., 1., 1.])

In [323]:
G_normalized = sum(blades)
G_normalized, U.sw(G_normalized)

(0.767 𝐞₁₂ + 0.487 𝐞₁₃ + -0.418 𝐞₁₄ + -0.418 𝐞₂₃ + -0.487 𝐞₂₄ + 0.767 𝐞₃₄,
 0.767 𝐞₁₂ + 0.487 𝐞₁₃ + -0.418 𝐞₁₄ + -0.418 𝐞₂₃ + -0.487 𝐞₂₄ + 0.767 𝐞₃₄)

In [324]:
# we can recover this "normalization"
sum(P(G_normalized, b) * n for b, n in zip(blades, blades_norm)), G

(0.309 𝐞₁₂ + 0.342 𝐞₁₃ + -0.269 𝐞₁₄ + -0.0283 𝐞₂₃ + -0.00432 𝐞₂₄ + 0.238 𝐞₃₄,
 0.309 𝐞₁₂ + 0.342 𝐞₁₃ + -0.269 𝐞₁₄ + -0.0283 𝐞₂₃ + -0.00432 𝐞₂₄ + 0.238 𝐞₃₄)

In [325]:
#7.21 zero if not onto the decomposed blades 
sympletic_basis = first_vecs + second_vecs
list(zip((v | (w | G_normalized) for v, w in product(sympletic_basis, repeat=2)), (product(['f', 'fp', 's', 'sp'], repeat=2))))

[(-1.11e-17, ('f', 'f')),
 (5.55e-17, ('f', 'fp')),
 (-1.0, ('f', 's')),
 (-5.55e-17, ('f', 'sp')),
 (-3.17e-17, ('fp', 'f')),
 (2.78e-17, ('fp', 'fp')),
 (1.67e-16, ('fp', 's')),
 (-1.0, ('fp', 'sp')),
 (1.0, ('s', 'f')),
 (-1.92e-16, ('s', 'fp')),
 (-2.22e-17, ('s', 's')),
 (9.91e-17, ('s', 'sp')),
 (-5.1e-17, ('sp', 'f')),
 (1.0, ('sp', 'fp')),
 (-5.55e-17, ('sp', 's')),
 (, ('sp', 'sp'))]

In [326]:
# Recover the normalization
[gm(v)/n for v, n in zip(sympletic_basis, blades_norm*2)], sympletic_basis

([-0.0966 𝐞₁ + 0.805 𝐞₂ + -0.585 𝐞₃,
  0.902 𝐞₁ + 0.108 𝐞₂ + -9.8e-17 𝐞₃ + -0.419 𝐞₄,
  -0.334 𝐞₁ + -0.318 𝐞₂ + -0.383 𝐞₃ + -0.8 𝐞₄,
  -0.258 𝐞₁ + 0.488 𝐞₂ + 0.715 𝐞₃ + -0.429 𝐞₄],
 [0.334 𝐞₁ + 0.318 𝐞₂ + 0.383 𝐞₃ + 0.8 𝐞₄,
  0.258 𝐞₁ + -0.488 𝐞₂ + -0.715 𝐞₃ + 0.429 𝐞₄,
  -0.0966 𝐞₁ + 0.805 𝐞₂ + -0.585 𝐞₃ + 2.78e-17 𝐞₄,
  0.902 𝐞₁ + 0.108 𝐞₂ + -0.419 𝐞₄])

In [327]:
#7.26a symplectic transformations of gm (isometries of skew bilinear form)
# conditions: unity and sympletic basis
translation = lambda x: (
    x + 
    2*(x|second_vecs[1]) * first_vecs[1] + 
    3*(x|second_vecs[0]) * first_vecs[0]
    )

outermorphism(translation, G_normalized, alg), G_normalized, outermorphism(translation, G, alg), G

(0.767 𝐞₁₂ + 0.487 𝐞₁₃ + -0.418 𝐞₁₄ + -0.418 𝐞₂₃ + -0.487 𝐞₂₄ + 0.767 𝐞₃₄,
 0.767 𝐞₁₂ + 0.487 𝐞₁₃ + -0.418 𝐞₁₄ + -0.418 𝐞₂₃ + -0.487 𝐞₂₄ + 0.767 𝐞₃₄,
 0.309 𝐞₁₂ + 0.342 𝐞₁₃ + -0.269 𝐞₁₄ + -0.0283 𝐞₂₃ + -0.00432 𝐞₂₄ + 0.238 𝐞₃₄,
 0.309 𝐞₁₂ + 0.342 𝐞₁₃ + -0.269 𝐞₁₄ + -0.0283 𝐞₂₃ + -0.00432 𝐞₂₄ + 0.238 𝐞₃₄)

In [328]:
#7.26b
rotation = lambda x: (
    x + 
    4*((x|first_vecs[1]) * second_vecs[1] + (x|second_vecs[0]) * first_vecs[0])
    )

outermorphism(rotation, G_normalized, alg), G_normalized, outermorphism(rotation, G, alg), G

(0.767 𝐞₁₂ + 0.487 𝐞₁₃ + -0.418 𝐞₁₄ + -0.418 𝐞₂₃ + -0.487 𝐞₂₄ + 0.767 𝐞₃₄,
 0.767 𝐞₁₂ + 0.487 𝐞₁₃ + -0.418 𝐞₁₄ + -0.418 𝐞₂₃ + -0.487 𝐞₂₄ + 0.767 𝐞₃₄,
 0.309 𝐞₁₂ + 0.342 𝐞₁₃ + -0.269 𝐞₁₄ + -0.0283 𝐞₂₃ + -0.00432 𝐞₂₄ + 0.238 𝐞₃₄,
 0.309 𝐞₁₂ + 0.342 𝐞₁₃ + -0.269 𝐞₁₄ + -0.0283 𝐞₂₃ + -0.00432 𝐞₂₄ + 0.238 𝐞₃₄)

In [329]:
#7.26c
outermorphism(lambda x: rotation(translation(x)), G, alg), G

(0.309 𝐞₁₂ + 0.342 𝐞₁₃ + -0.269 𝐞₁₄ + -0.0283 𝐞₂₃ + -0.00432 𝐞₂₄ + 0.238 𝐞₃₄,
 0.309 𝐞₁₂ + 0.342 𝐞₁₃ + -0.269 𝐞₁₄ + -0.0283 𝐞₂₃ + -0.00432 𝐞₂₄ + 0.238 𝐞₃₄)

In [343]:
#7.28
gp = sym_part(g, alg)
matrix = trans2matrix(gp, alg)
eigvalues, eigvecs = np.linalg.eigh(matrix)
eigvecs = [alg.vector(v) for v in eigvecs.transpose()]
neg_vecs, pos_vecs, null_vecs = [], [], []
for e, v in zip(eigvalues, eigvecs):
    if e < -1e-6:
        neg_vecs.append(v)
    elif e > 1e-6:
        pos_vecs.append(v)
    else:
        null_vecs.append(v)

neg_blade = wedge(neg_vecs)
nv_sign = (-1) ** len(neg_vecs) 
c = lambda x: nv_sign*neg_blade.sw(x)
h = lambda x: sum(np.sqrt(abs(e)) * P(x, v) for e, v in zip(eigvalues, eigvecs))

x | gp(b), h(x)|c(h(b)), c(x)|h(h(b))

(0.973, 0.973, 0.973)

In [344]:
#7.28 If the symmetric transformation is positive definite (Euclidean metric)
gpp = lambda x: sum(abs(e) * P(x, v) for e, v in zip(eigvalues, eigvecs))
hinv = lambda x: sum(1/np.sqrt(abs(e)) * P(x, v) for e, v in zip(eigvalues, eigvecs))
r = 2 # a arbitrary integer <= n
Q = random_r_blade(r, alg)
Q = Q / norm(Q)
f = lambda x: -hinv(Q.sw(h(x)))
x | gpp(b), f(x) | gpp(f(b)), x, f(x)

(0.941,
 0.941,
 0.852 𝐞₁ + 0.509 𝐞₂ + 0.0745 𝐞₃ + 0.0402 𝐞₄,
 1.37 𝐞₁ + 0.632 𝐞₂ + -1.74 𝐞₃ + 1.38 𝐞₄)

In [345]:
#7.33 isometry of c
def ortho_split(x, A):
    xp = P(x, A)
    return xp, x-xp

xp, xo = ortho_split(x, neg_blade)
bp, bo = ortho_split(b, neg_blade)
x | c(b), (xo | bo) - (xp | bp)


(0.904, 0.904)

In [350]:
# Let R be a rotor (see next section) preserves the reflected subspace
R = 1
if len(neg_vecs) >= 2:
    R *= blade_exp(3.3 * wedge(neg_vecs[:2]))
if len(pos_vecs) >= 2:
    R *= blade_exp(5.5 * wedge(pos_vecs[:2]))
R.sw(neg_blade), neg_blade

(0.144 𝐞₁₂ + 0.115 𝐞₁₃ + -0.455 𝐞₁₄ + -0.277 𝐞₂₃ + 0.79 𝐞₂₄ + -0.242 𝐞₃₄,
 0.144 𝐞₁₂ + 0.115 𝐞₁₃ + -0.455 𝐞₁₄ + -0.277 𝐞₂₃ + 0.79 𝐞₂₄ + -0.242 𝐞₃₄)

In [347]:
# an isometry of a pseudoEuclidean metric
f = lambda x: -hinv(R.sw(h(x)))
x | gp(b), f(x) | gp(f(b)), x, f(x)

(0.973,
 0.973,
 0.852 𝐞₁ + 0.509 𝐞₂ + 0.0745 𝐞₃ + 0.0402 𝐞₄,
 -0.864 𝐞₁ + -0.48 𝐞₂ + -0.0741 𝐞₃ + -0.0665 𝐞₄)

So a rotation preserves the reflected subspace is an isometry, any other isometry? The next section encode the metric into the algebra. So an orthogonal transform becomes an isometry of the vector space.

In [353]:
# signature
[v | c(v) for v in eigvecs]

[-1.0, -1.0, 1.0, 1.0]

In [380]:
p, q, r = 0,0,0
for e in eigvalues:
    if e > 1e-6:
        p += 1
    elif e < -1e-6:
        q += 1
    else:
        r += 1
p, q, r

(2, 2, 0)

In [381]:
#8.2 Say the reflected subspace is 1d
alg = Algebra(3, 1)
locals().update(alg.blades)
x, y, u = random_r_vectors(3, alg)
u = u / norm(u)
R = lambda x: - u.sw(x)
R(x) | R(y), x | y

(0.857, 0.857)

Cartan-Dieudonne theorem: An isometry is a composition of at most d reflections. Identifying invariants of isometries as elements of geometry leads to modern developments.

In [382]:
#8.3 Cartan-Dieudonne theorem: An isometry is a composition of at most d reflections.

k = 3 # k <= d
vectors = random_r_vectors(k, alg)
p = sum((v**2)[0] > 0 for v in vectors)
U = gprod(vectors) # a versor, of either even or odd grades
f = lambda x: (-1) ** k * U * x / U
f_ = lambda x: (-1) ** p * U.sw(x) / normsq(U) #8.8b
f(x) | f(y), x | y, f_(x) | f_(y)

(0.857 + -4.97e-16 𝐞₁₂ + -5.04e-16 𝐞₁₃ + -6.26e-16 𝐞₁₄ + -8.27e-16 𝐞₂₃ + -6.71e-17 𝐞₂₄ + 9.6e-16 𝐞₃₄,
 0.857,
 0.857)

In [383]:
#8.8
def random_versor(r, alg):
    return gprod(random_r_vectors(r, alg))

def random_versor_(r, alg):
    vectors = random_r_vectors(r, alg)
    p = sum((v**2)[0] > 0 for v in vectors)
    return gprod(vectors), p
k = 3
for _ in range(10):
    U, p = random_versor_(k, alg)
    U = U/norm(U)
    print(k - p, U.reverse() * U)

1 -1.0 + 4.16e-17 𝐞₁₃ + 2.22e-16 𝐞₁₄ + 4.44e-16 𝐞₂₄ + 4.44e-16 𝐞₃₄ + 4.44e-16 𝐞₁₂₃₄
1 -1.0 + 2.08e-17 𝐞₁₃ + 5.56e-19 𝐞₁₄ + 2.78e-17 𝐞₂₃ + 3.47e-18 𝐞₂₄ + -3.47e-18 𝐞₃₄ + 3.47e-17 𝐞₁₂₃₄
0 1.0 + -5.55e-17 𝐞₁₃ + -4.86e-17 𝐞₁₄ + -1.11e-16 𝐞₂₄ + 4.44e-16 𝐞₁₂₃₄
1 -1.0 + -2.66e-15 𝐞₁₄ + -2.66e-15 𝐞₂₄ + 3.55e-15 𝐞₃₄ + 1.87e-14 𝐞₁₂₃₄
0 1.0 + -4.55e-18 𝐞₁₃ + 5.59e-17 𝐞₁₄ + -2.43e-17 𝐞₂₃ + -2.08e-17 𝐞₂₄ + -4.68e-17 𝐞₃₄ + 1.21e-16 𝐞₁₂₃₄
1 -1.0 + 2.22e-16 𝐞₁₂ + 2.22e-16 𝐞₂₄ + 5.55e-17 𝐞₃₄
0 1.0 + -2.78e-17 𝐞₁₃ + 1.73e-17 𝐞₁₄ + -5.2e-18 𝐞₂₃ + 3.47e-17 𝐞₂₄ + -1.39e-16 𝐞₁₂₃₄
0 1.0 + 1.28e-17 𝐞₁₃ + 2.34e-17 𝐞₁₄ + -1.04e-17 𝐞₂₃ + 9.97e-17 𝐞₂₄ + -1.39e-17 𝐞₃₄ + 2.34e-17 𝐞₁₂₃₄
1 -1.0 + 1.87e-14 𝐞₁₃ + -1.03e-14 𝐞₁₄ + -7.11e-15 𝐞₂₃ + -3.55e-15 𝐞₂₄ + 1.42e-14 𝐞₃₄ + -2.49e-14 𝐞₁₂₃₄
0 1.0 + -1.11e-16 𝐞₁₃ + -6.94e-17 𝐞₁₄ + 2.22e-16 𝐞₂₄ + -2.22e-16 𝐞₁₂₃₄


In [384]:
# versors (Pin) are 2:1 homomorphisc to orthogonal transformations (O)
U = random_versor(3, alg)
U.sw(x), (-U).sw(x)

(-0.167 𝐞₁ + -0.014 𝐞₂ + 0.198 𝐞₃ + 0.257 𝐞₄,
 -0.167 𝐞₁ + -0.014 𝐞₂ + 0.198 𝐞₃ + 0.257 𝐞₄)

In [385]:
#8.5
I = alg.pseudoscalar((1,))
I.sw(x), x

(0.67 𝐞₁ + 0.387 𝐞₂ + 0.642 𝐞₃ + 0.322 𝐞₄,
 0.67 𝐞₁ + 0.387 𝐞₂ + 0.642 𝐞₃ + 0.322 𝐞₄)

In [386]:
#8.7 orthogonal transformations (O) with determinant 1 (SO) correspond to even versors (Spin)
for k in range(1,5):
    U = gprod(random_r_vectors(k, alg)) # a versor, of either even or odd grades
    f = lambda x: (-1) ** (k*alg.d) * U.sw(x)/normsq(U)
    print(k, outermorphism(f, I, alg))

1 -1.0 𝐞₁₂₃₄
2 1.0 𝐞₁₂₃₄
3 -1.0 𝐞₁₂₃₄
4 1.0 𝐞₁₂₃₄


In [387]:
#8.9 rotors (Spin+) are a part of even versors (Spin), correpond to a part of SO as rotations (SO+)
def random_even_versor(alg):
    d = alg.d
    if (alg.d % 2):
        d -= 1
    return gprod(random_r_vectors(d, alg))


def random_rotor(alg, max_iter=100):
    for _ in range(max_iter):
        S = random_even_versor(alg)
        sign  = (S.reverse().sp(S))[0]
        if sign > 0:
            return S/norm(S)
        

def random_rotor_count(alg, max_iter=100):
    count = 0
    for _ in range(max_iter):
        S = random_even_versor(alg)
        sign  = (S.reverse().sp(S))[0]
        if sign > 0:
            count += 1
    return count

[(p, q, r, random_rotor_count(Algebra(p, q, r), 2000)/2000) for p, q, r in ((3,1,0),(1,3,0),(4,1,0),(3,0,1),(2,2,0))]

[(3, 1, 0, 0.6445),
 (1, 3, 0, 0.645),
 (4, 1, 0, 0.7845),
 (3, 0, 1, 1.0),
 (2, 2, 0, 0.498)]

versors (Pin) are products of {1, a, b, ab} and a rotor

In [388]:
#8.12
def random_simple_rotor(alg):
    while True:
        S = random_versor(2, alg)
        sign  = (S.reverse().sp(S))[0]
        if sign > 0:
            return S/norm(S)

In [389]:
#8.13
S = random_simple_rotor(alg)
u = x | S.grade(2)
u = u/norm(u)
v = S.sw(u)
v, S ** 2 * u, v ** 2, u ** 2, v * u, S ** 2

(-0.499 𝐞₁ + -1.41 𝐞₂ + -0.443 𝐞₃ + -1.2 𝐞₄,
 -0.499 𝐞₁ + -1.41 𝐞₂ + -0.443 𝐞₃ + -1.2 𝐞₄ + 2.73e-16 𝐞₁₂₃ + 1.73e-17 𝐞₁₂₄ + 3.99e-16 𝐞₁₃₄ + -3e-17 𝐞₂₃₄,
 1.0,
 1.0,
 -0.9 + -0.737 𝐞₁₂ + 0.0633 𝐞₁₃ + -0.318 𝐞₁₄ + 0.835 𝐞₂₃ + 0.873 𝐞₂₄ + -0.435 𝐞₃₄,
 -0.9 + -0.737 𝐞₁₂ + 0.0633 𝐞₁₃ + -0.318 𝐞₁₄ + 0.835 𝐞₂₃ + 0.873 𝐞₂₄ + -0.435 𝐞₃₄ + 4.44e-16 𝐞₁₂₃₄)

In [390]:
S ** 2 * (S ** 2).reverse(), S * S.reverse()

(1.0 + -2.84e-17 𝐞₁₃ + 1.07e-16 𝐞₁₄ + 3.04e-17 𝐞₂₃ + -1.39e-16 𝐞₂₄ + 5.73e-18 𝐞₃₄ + -1.68e-15 𝐞₁₂₃₄,
 1.0 + -1.94e-16 𝐞₁₄ + 2.22e-16 𝐞₂₄ + 1.67e-16 𝐞₃₄ + -4.44e-16 𝐞₁₂₃₄)

In [391]:
#8.15
(S ** 2).reverse(), u * v, v * u

(-0.9 + 0.737 𝐞₁₂ + -0.0633 𝐞₁₃ + 0.318 𝐞₁₄ + -0.835 𝐞₂₃ + -0.873 𝐞₂₄ + 0.435 𝐞₃₄ + 4.44e-16 𝐞₁₂₃₄,
 -0.9 + 0.737 𝐞₁₂ + -0.0633 𝐞₁₃ + 0.318 𝐞₁₄ + -0.835 𝐞₂₃ + -0.873 𝐞₂₄ + 0.435 𝐞₃₄,
 -0.9 + -0.737 𝐞₁₂ + 0.0633 𝐞₁₃ + -0.318 𝐞₁₄ + 0.835 𝐞₂₃ + 0.873 𝐞₂₄ + -0.435 𝐞₃₄)

In [392]:
#8.16
S2 = S ** 2
blade = S2.grade(2)
bladesq = (blade ** 2)
bladesq

-0.19 + 2.78e-17 𝐞₁₃ + 4.16e-17 𝐞₁₄ + 8.88e-16 𝐞₁₂₃₄

In [460]:
# 3 types of rotors (Spin+)
def type_convert(n):
    if n > 1e-6:
        return 1
    elif n < -1e-6:
        return -1
    else:
        return 0

# Unfortunately parabolic rotors are of measure zero, 
# so this function has zero chance generating them
def random_simple_rotor(alg, signature=None, max_iter=100):
    for _ in range(max_iter):
        S = random_versor(2, alg)
        if signature is not None:
            if type_convert((S.grade(2) ** 2)[0]) != signature:
                continue
        sign  = (S.reverse().sp(S))[0]
        if sign > 0:
            return S/norm(S)
        
[random_simple_rotor(alg, signature=i) for i in range(-1, 2)]

[0.201 + 0.516 𝐞₁₂ + 1.11 𝐞₁₃ + 0.617 𝐞₁₄ + -0.159 𝐞₂₃ + -0.254 𝐞₂₄ + -0.358 𝐞₃₄,
 None,
 1.52 + 0.902 𝐞₁₂ + 1.07 𝐞₁₃ + 1.69 𝐞₁₄ + 0.857 𝐞₂₃ + 0.985 𝐞₂₄ + -0.435 𝐞₃₄]

In [461]:
def simple_rotor_log(R: MultiVector, tol=1e-6):
    blade = R.grade(2)
    signature = (blade ** 2)[0]
    if signature > tol:
        return np.arccosh(R[0]) / norm(blade) * blade
    if signature < -tol:
        return np.arccos(R[0]) / norm(blade) * blade
    else:
        return blade

In [462]:
#8.17a elliptic rotor (rotation), run several times you can see sqrt allows a sign change
S = random_simple_rotor(alg, signature=-1)
S2 = S ** 2
b = simple_rotor_log(S2)
S2, blade_exp(b), blade_exp(b - 6*np.pi*b/norm(b)), S, blade_exp(b/2), simple_rotor_sqrt(S2), blade_exp(3.3*b).sw(S)

(0.638 + 1.06 𝐞₁₂ + 1.05 𝐞₁₃ + -0.17 𝐞₁₄ + 0.0318 𝐞₂₃ + -0.9 𝐞₂₄ + -0.893 𝐞₃₄ + -5.55e-17 𝐞₁₂₃₄,
 0.638 + 1.06 𝐞₁₂ + 1.05 𝐞₁₃ + -0.17 𝐞₁₄ + 0.0318 𝐞₂₃ + -0.9 𝐞₂₄ + -0.893 𝐞₃₄,
 0.638 + 1.06 𝐞₁₂ + 1.05 𝐞₁₃ + -0.17 𝐞₁₄ + 0.0318 𝐞₂₃ + -0.9 𝐞₂₄ + -0.893 𝐞₃₄,
 0.905 + 0.584 𝐞₁₂ + 0.583 𝐞₁₃ + -0.0942 𝐞₁₄ + 0.0175 𝐞₂₃ + -0.497 𝐞₂₄ + -0.493 𝐞₃₄,
 0.905 + 0.584 𝐞₁₂ + 0.583 𝐞₁₃ + -0.0942 𝐞₁₄ + 0.0175 𝐞₂₃ + -0.497 𝐞₂₄ + -0.493 𝐞₃₄,
 0.905 + 0.584 𝐞₁₂ + 0.583 𝐞₁₃ + -0.0942 𝐞₁₄ + 0.0175 𝐞₂₃ + -0.497 𝐞₂₄ + -0.493 𝐞₃₄ + -3.07e-17 𝐞₁₂₃₄,
 0.905 + 0.584 𝐞₁₂ + 0.583 𝐞₁₃ + -0.0942 𝐞₁₄ + 0.0175 𝐞₂₃ + -0.497 𝐞₂₄ + -0.493 𝐞₃₄ + -5.08e-17 𝐞₁₂₃₄)

In [463]:
#8.17b hyperbolic rotor (boost)
S = random_simple_rotor(alg, signature=1)
S2 = S ** 2
b = simple_rotor_log(S2)
S2, blade_exp(b), S, blade_exp(b/2), simple_rotor_sqrt(S2), blade_exp(3.3*b).sw(S)

(1.44 + -0.544 𝐞₁₂ + 1.41 𝐞₁₃ + -1.2 𝐞₁₄ + 1.28 𝐞₂₃ + -0.379 𝐞₂₄ + -1.84 𝐞₃₄ + 6.38e-16 𝐞₁₂₃₄,
 1.44 + -0.544 𝐞₁₂ + 1.41 𝐞₁₃ + -1.2 𝐞₁₄ + 1.28 𝐞₂₃ + -0.379 𝐞₂₄ + -1.84 𝐞₃₄,
 1.1 + -0.247 𝐞₁₂ + 0.637 𝐞₁₃ + -0.544 𝐞₁₄ + 0.579 𝐞₂₃ + -0.171 𝐞₂₄ + -0.834 𝐞₃₄,
 1.1 + -0.247 𝐞₁₂ + 0.637 𝐞₁₃ + -0.544 𝐞₁₄ + 0.579 𝐞₂₃ + -0.171 𝐞₂₄ + -0.834 𝐞₃₄,
 1.1 + -0.247 𝐞₁₂ + 0.637 𝐞₁₃ + -0.544 𝐞₁₄ + 0.579 𝐞₂₃ + -0.171 𝐞₂₄ + -0.834 𝐞₃₄ + 2.89e-16 𝐞₁₂₃₄,
 1.1 + -0.247 𝐞₁₂ + 0.637 𝐞₁₃ + -0.544 𝐞₁₄ + 0.579 𝐞₂₃ + -0.171 𝐞₂₄ + -0.834 𝐞₃₄ + -2.84e-13 𝐞₁₂₃₄)

In [464]:
#8.17c 8.20 parabolic rotor (translation)
S = (e2 + e3 - e4) * e2
S2 = S ** 2
b = simple_rotor_log(S2)
S2, blade_exp(b), S, blade_exp(b/2), simple_rotor_sqrt(S2), blade_exp(3.3*b).sw(S)

(1 + -2 𝐞₂₃ + 2 𝐞₂₄,
 1 + -2 𝐞₂₃ + 2 𝐞₂₄,
 1 + -1 𝐞₂₃ + 1 𝐞₂₄,
 1 + -1.0 𝐞₂₃ + 1.0 𝐞₂₄,
 1.0 + -1.0 𝐞₂₃ + 1.0 𝐞₂₄,
 1.0 + -1.0 𝐞₂₃ + 1.0 𝐞₂₄)

In [416]:
#8.21
B = S.grade(2) / (1 + S.grade(0))
B, (1+B)**2/(normsq(1+B)), S

(-0.5 𝐞₂₃ + 0.5 𝐞₂₄, 1.0 + -1.0 𝐞₂₃ + 1.0 𝐞₂₄, 1 + -1 𝐞₂₃ + 1 𝐞₂₄)

In [401]:
#8.25
R, u = random_rotor(alg), random_r_blade(1, alg)
v = R.sw(u)
S = simple_rotor_sqrt(v * u / normsq(u))
U = S.reverse() * R
R, S * U, S * u, u * S.reverse(), U * u, u * U

(0.545 + -3.75 𝐞₁₂ + 0.1 𝐞₁₃ + 0.0983 𝐞₁₄ + 2.35 𝐞₂₃ + 4.34 𝐞₂₄ + -0.0145 𝐞₃₄ + -0.273 𝐞₁₂₃₄,
 -0.545 + 3.75 𝐞₁₂ + -0.1 𝐞₁₃ + -0.0983 𝐞₁₄ + -2.35 𝐞₂₃ + -4.34 𝐞₂₄ + 0.0145 𝐞₃₄ + 0.273 𝐞₁₂₃₄,
 -8.66 𝐞₁ + 1.58 𝐞₂ + -4.02 𝐞₃ + -9.72 𝐞₄ + -8.88e-16 𝐞₁₂₄ + -1.33e-15 𝐞₁₃₄ + -4.44e-16 𝐞₂₃₄,
 -8.66 𝐞₁ + 1.58 𝐞₂ + -4.02 𝐞₃ + -9.72 𝐞₄ + 8.88e-16 𝐞₁₂₄ + 1.33e-15 𝐞₁₃₄ + 4.44e-16 𝐞₂₃₄,
 0.638 𝐞₁ + 0.624 𝐞₂ + 0.789 𝐞₃ + 0.746 𝐞₄ + -2.79 𝐞₁₂₃ + -0.17 𝐞₁₂₄ + -0.0222 𝐞₁₃₄ + -3.08 𝐞₂₃₄,
 0.638 𝐞₁ + 0.624 𝐞₂ + 0.789 𝐞₃ + 0.746 𝐞₄ + -2.79 𝐞₁₂₃ + -0.17 𝐞₁₂₄ + -0.0222 𝐞₁₃₄ + -3.08 𝐞₂₃₄)

In [465]:
#8.10 a spinor is an even versor for odd dimension (a Dirac representation);
# a sum of an even versor and its dual for even dimension (two Weyl representations)
def random_study_number(alg):
    a, b = np.random.rand(2)
    return (a + b*alg.pseudoscalar((1,)))

def random_spinor(alg: Algebra):
    if alg.d % 4:
        return random_even_versor(alg)
    return random_study_number(alg) * random_even_versor(alg)

random_spinor(Algebra(3)), random_spinor(Algebra(4))


(1.39 + 0.0255 𝐞₁₂ + -0.0102 𝐞₁₃ + -0.111 𝐞₂₃,
 0.349 + 0.228 𝐞₁₂ + -0.51 𝐞₁₃ + 0.234 𝐞₁₄ + 0.878 𝐞₂₃ + -0.282 𝐞₂₄ + -0.0916 𝐞₃₄ + 1.03 𝐞₁₂₃₄)

In [491]:
# A pointor depends on a point
def random_point(alg: Algebra):
    return random_r_blade(alg.d - 1, alg)

O = random_point(alg)

# point reflection on the point itself is a pointor
terms_ratio(O, O.sw(O))

array([-98.05811122, -98.05811122, -98.05811122, -98.05811122])

In [528]:
# versors not crossing O-subspace are pointers 
vectors = blade_split(O, alg)
B1, B2 = wedge(vectors[:2]), wedge(vectors[1:])
Od = O.dual()
terms_ratio(O, Od.sw(O)), terms_ratio(B1.sw(O), O), terms_ratio(vectors[0].sw(O), O), 

(array([-49.74442464, -49.74442464, -49.74442464, -49.74442464]),
 array([12.71156251, 12.71156251, 12.71156251, 12.71156251]),
 array([17.1044585, 17.1044585, 17.1044585, 17.1044585]))

In [530]:
# rotors with reflections are pointers
R1 = blade_exp(3.3*B1)
R2 = blade_exp(4.4*B2)
terms_ratio((Od * R1 * R2).sw(O), O), terms_ratio((Od * vectors[2] * R1).sw(O), O)

(array([-0.02010276, -0.02010276, -0.02010276, -0.02010276]),
 array([0.02068227, 0.02068227, 0.02068227, 0.02068227]))

In [500]:
# rotors cross O-subspace may not be pointers
terms_ratio(blade_exp(Od ^ vectors[2]).sw(O), O)

array([0.13246279, 0.97240152, 0.97240152, 0.97240152])

In [531]:
# A general pointor suggested by "From Invariant Decomposition to Spinors"?
# How do we find the point specified by a pointor?
def random_odd_versor(alg):
    d = alg.d
    if not (alg.d % 2):
        d -= 1
    return gprod(random_r_vectors(d, alg))


# Actually I'm not sure these generate pointors
# def random_pointor(alg: Algebra):
#     return random_odd_versor(alg) * random_even_versor(alg)
    
# def random_pointor(alg: Algebra):
#     v = random_r_blade(1, alg)
#     return (np.random.rand() + v) * random_even_versor(alg)

def random_pointor(alg: Algebra):
    v = random_r_blade(1, alg)
    return (np.random.rand() + v) * random_versor(2, alg)

P = random_pointor(alg)
P

0.744 + 1.21 𝐞₁ + 0.868 𝐞₂ + 1.0 𝐞₃ + 1.45 𝐞₄ + 0.0815 𝐞₁₂ + 0.711 𝐞₁₃ + 0.741 𝐞₁₄ + 0.135 𝐞₂₃ + 0.165 𝐞₂₄ + 0.212 𝐞₃₄ + -0.5 𝐞₁₂₃ + -0.436 𝐞₁₂₄ + 0.736 𝐞₁₃₄ + 0.289 𝐞₂₃₄

In [532]:
# the point-grade
O = P.grade(3)
terms_ratio(P.sw(O), O)

array([1.01708123, 1.01708123, 1.01708123, 1.01708123])